In [1]:
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from fake_useragent import UserAgent
import pandas as pd
import time

In [2]:
def get_data(url):
    user_agent = UserAgent()
    headers = {
        "user-agent": f"{user_agent.random}"
    }
    item = 1
    req = requests.get(url + f"{item}", headers)
    while req.text != "{\"message\":\"Такой страницы не найдено\"}":
        print("----------------page - ", item, "---------------")
        data = json.loads(req.text)
        urls = []
        for article in data:
            urls.append(article["url"])
        print("URLS - ", len(urls))
        url1 = "https://postnauka.ru"
        cntr = 1
        for i in urls:
            try:
                print("URL NUMBER - ", cntr)
                cntr += 1
                options = webdriver.ChromeOptions()
                options.add_argument(f"user-agent={user_agent.random}")
                options.add_argument("headless")
                options.add_argument("window-size=1920x935")
                driver = webdriver.Chrome(options=options)
                driver.get(url1 + f"{i}")
                time.sleep(1)
                html = driver.page_source
                driver.close()
                soup = BeautifulSoup(html, "lxml")
                texts = soup.find_all("div", {"class": "p-text"})
                text = ""
                for j in texts:
                    text += j.text
                    text += "\n"
                if len(text) < 2:
                    continue
                #print(text)
                df = pd.DataFrame(
                    {
                        "texts": text,
                    },
                    index=[0],
                )
                df.columns = ["texts"]
                print(df)
                df.to_csv("literature.csv", index=False, mode="a", header=False)
            except():
                continue
        item += 1
        req = requests.get(url + f"{item}", headers)
    return 0


In [3]:
get_data("https://postnauka.ru/api/v4/grid/literature?page=")

----------------page -  1 ---------------
URLS -  40
URL NUMBER -  1
                                               texts
0  Сначала граф Лев Николаевич Толстой отказался ...
URL NUMBER -  2
                                               texts
0  История о том, как Тургенев поругался с Достое...
URL NUMBER -  3
                                               texts
0  В рамках курса «Русский политический роман» по...
URL NUMBER -  4
                                               texts
0  Гость Рубки ПостНауки — Фёдор Успенский, докто...
URL NUMBER -  5
                                               texts
0  Гость Рубки ПостНауки — Николай Александров, к...
URL NUMBER -  6
                                               texts
0  Литературоведы считают, что Шекспир во многом ...
URL NUMBER -  7
                                               texts
0  Уильям Шекспир — всемирно известный английский...
URL NUMBER -  8
                                               texts
0  Первые исследования «

KeyboardInterrupt: 